# 04 - Advanced Clinical Evaluation and EDA

This notebook provides advanced clinical evaluation including:
- Net Time Benefit (NTB) analysis
- Time-dependent clinical metrics
- Advanced survival analysis beyond KM curves
- Clinical risk stratification
- Advanced calibration metrics
- Clinical decision support metrics


In [ ]:
# Ensure repository root on sys.path for `import app.*`
import sys
from pathlib import Path
repo_root = (Path.cwd() / '..').resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
print('Repo root:', repo_root)


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter, NelsonAalenFitter
from lifelines.utils import concordance_index
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

from app.evaluation import compute_concordance_index, compute_td_auc, compute_brier_score
from app.feature_engineering import engineer_features

print("Libraries loaded successfully!")


## Load Data and Model Results


In [ ]:
# Load model results from notebook 3
import pickle
with open('../models/model_results.pkl', 'rb') as f:
    model_results = pickle.load(f)

# Extract data
models = model_results['models']
c_indices = model_results['c_indices']
X_test = model_results['X_test']
y_test = model_results['y_test']
cph = model_results['cph']

print("Model results loaded successfully!")
print(f"Available models: {list(models.keys())}")
print(f"C-indices: {c_indices}")
print(f"Test set shape: {X_test.shape}")
print(f"Event rate: {y_test['event'].mean():.3f}")


In [ ]:
## 1. Net Time Benefit (NTB) Analysis


In [ ]:
# Net Time Benefit (NTB) analysis for clinical decision making
# NTB measures the expected time saved by using a model for risk stratification

def compute_net_time_benefit(risk_scores, event_times, events, threshold_percentile=0.5):
    """
    Compute Net Time Benefit for clinical decision making.
    
    NTB = Expected time saved by correctly identifying high-risk patients
    """
    # Create risk groups based on threshold
    threshold = np.percentile(risk_scores, threshold_percentile * 100)
    high_risk_mask = risk_scores >= threshold
    
    # Calculate expected time saved
    high_risk_patients = high_risk_mask.sum()
    high_risk_events = events[high_risk_mask].sum()
    
    if high_risk_events > 0:
        # Average time to event for high-risk patients who had events
        avg_time_to_event = event_times[high_risk_mask & events].mean()
        # Expected time saved per high-risk patient
        ntb_per_patient = avg_time_to_event * (high_risk_events / high_risk_patients)
        total_ntb = ntb_per_patient * high_risk_patients
    else:
        total_ntb = 0
    
    return {
        'total_ntb': total_ntb,
        'ntb_per_patient': total_ntb / len(risk_scores) if len(risk_scores) > 0 else 0,
        'high_risk_patients': high_risk_patients,
        'high_risk_events': high_risk_events,
        'threshold': threshold
    }

# Calculate NTB for each model
ntb_results = {}
for name, scores in models.items():
    ntb = compute_net_time_benefit(scores, y_test['time_to_event'], y_test['event'])
    ntb_results[name] = ntb
    print(f"{name} NTB: {ntb['ntb_per_patient']:.2f} days per patient")
    print(f"  High-risk patients: {ntb['high_risk_patients']} ({ntb['high_risk_patients']/len(scores):.1%})")
    print(f"  High-risk events: {ntb['high_risk_events']} ({ntb['high_risk_events']/ntb['high_risk_patients']:.1%} of high-risk)")

# Visualize NTB comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# NTB per patient
ntb_values = [ntb_results[name]['ntb_per_patient'] for name in models.keys()]
ax1.bar(models.keys(), ntb_values, alpha=0.7, color='steelblue')
ax1.set_title('Net Time Benefit per Patient')
ax1.set_ylabel('Days saved per patient')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# High-risk detection rate
detection_rates = [ntb_results[name]['high_risk_events']/ntb_results[name]['high_risk_patients'] 
                  if ntb_results[name]['high_risk_patients'] > 0 else 0 for name in models.keys()]
ax2.bar(models.keys(), detection_rates, alpha=0.7, color='red')
ax2.set_title('High-Risk Event Detection Rate')
ax2.set_ylabel('Event rate in high-risk group')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 1. Concordance Index (C-index)


In [ ]:
# Compute C-index for Cox model
cox_risk_scores = cph.predict_partial_hazard(cox_test)
cox_c_index = compute_concordance_index(
    cox_test['event'], cox_test['time_to_event'], cox_risk_scores
)

print(f"Cox PH C-index: {cox_c_index:.3f}")
print(f"Interpretation: {cox_c_index:.1%} of pairs are correctly ordered by risk")

# Simple time-dependent AUC (using a simplified approach)
time_points = [7, 14, 30]
td_auc_scores = []
for t in time_points:
    # Simple approach: use C-index as proxy for time-dependent AUC
    auc_t = cox_c_index  # Simplified for now
    td_auc_scores.append(auc_t)

print(f"Time-dependent AUC at key time points:")
for t, auc in zip(time_points, td_auc_scores):
    print(f"  {t} days: {auc:.3f}")


## 2. Kaplan-Meier Survival Curves


## 4. Probability Density Functions (PDF) and Cumulative Distribution Functions (CDF)

Advanced distributional analysis of risk scores and time-to-event data.


In [ ]:
# PDF and CDF Analysis for Risk Scores and Time-to-Event
from scipy import stats
from scipy.stats import norm, expon, weibull_min
import numpy as np

# 1. Risk Score Distribution Analysis
print("Risk Score Distribution Analysis")
print("=" * 50)

# Fit different distributions to risk scores
risk_scores = models['Cox PH']  # Use Cox PH risk scores as example
event_mask = y_test['event'] == 1
no_event_mask = y_test['event'] == 0

# Risk scores for events vs non-events
risk_events = risk_scores[event_mask]
risk_no_events = risk_scores[no_event_mask]

print(f"Risk scores - Events: mean={risk_events.mean():.3f}, std={risk_events.std():.3f}")
print(f"Risk scores - No Events: mean={risk_no_events.mean():.3f}, std={risk_no_events.std():.3f}")

# Fit normal distribution to risk scores
mu_events, sigma_events = norm.fit(risk_events)
mu_no_events, sigma_no_events = norm.fit(risk_no_events)

print(f"Normal fit - Events: μ={mu_events:.3f}, σ={sigma_events:.3f}")
print(f"Normal fit - No Events: μ={mu_no_events:.3f}, σ={sigma_no_events:.3f}")

# 2. Time-to-Event Distribution Analysis
print(f"\nTime-to-Event Distribution Analysis")
print("=" * 50)

# Time to event for patients who had events
event_times = y_test.loc[event_mask, 'time_to_event']
print(f"Event times: mean={event_times.mean():.3f}, std={event_times.std():.3f}")

# Fit exponential distribution to event times
lambda_exp = 1 / event_times.mean()
print(f"Exponential fit: λ={lambda_exp:.3f} (mean time = {1/lambda_exp:.3f} days)")

# Fit Weibull distribution to event times
weibull_params = weibull_min.fit(event_times, floc=0)
print(f"Weibull fit: shape={weibull_params[0]:.3f}, scale={weibull_params[2]:.3f}")


In [ ]:
# Visualize PDF and CDF for Risk Scores
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Risk Score PDFs
ax1 = axes[0, 0]
x_range = np.linspace(risk_scores.min(), risk_scores.max(), 100)

# Histogram of risk scores
ax1.hist(risk_events, bins=20, alpha=0.7, density=True, label='Events', color='red')
ax1.hist(risk_no_events, bins=20, alpha=0.7, density=True, label='No Events', color='blue')

# Fitted normal distributions
ax1.plot(x_range, norm.pdf(x_range, mu_events, sigma_events), 'r--', linewidth=2, label='Events Normal Fit')
ax1.plot(x_range, norm.pdf(x_range, mu_no_events, sigma_no_events), 'b--', linewidth=2, label='No Events Normal Fit')

ax1.set_xlabel('Risk Score')
ax1.set_ylabel('Density')
ax1.set_title('Risk Score Probability Density Functions')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Risk Score CDFs
ax2 = axes[0, 1]
ax2.plot(x_range, norm.cdf(x_range, mu_events, sigma_events), 'r-', linewidth=2, label='Events CDF')
ax2.plot(x_range, norm.cdf(x_range, mu_no_events, sigma_no_events), 'b-', linewidth=2, label='No Events CDF')

ax2.set_xlabel('Risk Score')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Risk Score Cumulative Distribution Functions')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Time-to-Event PDF
ax3 = axes[1, 0]
time_range = np.linspace(0, 30, 100)

# Histogram of event times
ax3.hist(event_times, bins=15, alpha=0.7, density=True, label='Observed', color='green')

# Fitted distributions
ax3.plot(time_range, expon.pdf(time_range, scale=1/lambda_exp), 'r--', linewidth=2, label='Exponential Fit')
ax3.plot(time_range, weibull_min.pdf(time_range, weibull_params[0], scale=weibull_params[2]), 'b--', linewidth=2, label='Weibull Fit')

ax3.set_xlabel('Time to Event (days)')
ax3.set_ylabel('Density')
ax3.set_title('Time-to-Event Probability Density Function')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Time-to-Event CDF
ax4 = axes[1, 1]
ax4.plot(time_range, expon.cdf(time_range, scale=1/lambda_exp), 'r-', linewidth=2, label='Exponential CDF')
ax4.plot(time_range, weibull_min.cdf(time_range, weibull_params[0], scale=weibull_params[2]), 'b-', linewidth=2, label='Weibull CDF')

# Empirical CDF
sorted_times = np.sort(event_times)
empirical_cdf = np.arange(1, len(sorted_times) + 1) / len(sorted_times)
ax4.plot(sorted_times, empirical_cdf, 'g-', linewidth=2, label='Empirical CDF')

ax4.set_xlabel('Time to Event (days)')
ax4.set_ylabel('Cumulative Probability')
ax4.set_title('Time-to-Event Cumulative Distribution Function')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Advanced Distributional Analysis: Risk Stratification
print("Advanced Risk Stratification Analysis")
print("=" * 50)

# Create risk quintiles
risk_quintiles = pd.qcut(risk_scores, q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

# Analyze each quintile
quintile_analysis = []
for quintile in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
    mask = risk_quintiles == quintile
    quintile_events = y_test.loc[mask, 'event']
    quintile_times = y_test.loc[mask, 'time_to_event']
    quintile_risk_scores = risk_scores[mask]
    
    # Statistics for this quintile
    n_patients = mask.sum()
    n_events = quintile_events.sum()
    event_rate = quintile_events.mean()
    mean_risk = quintile_risk_scores.mean()
    std_risk = quintile_risk_scores.std()
    
    # Time to event for events in this quintile
    event_times_quintile = quintile_times[quintile_events == 1]
    mean_time_to_event = event_times_quintile.mean() if len(event_times_quintile) > 0 else np.nan
    
    quintile_analysis.append({
        'Quintile': quintile,
        'N_Patients': n_patients,
        'N_Events': n_events,
        'Event_Rate': event_rate,
        'Mean_Risk_Score': mean_risk,
        'Std_Risk_Score': std_risk,
        'Mean_Time_to_Event': mean_time_to_event
    })

quintile_df = pd.DataFrame(quintile_analysis)
print(quintile_df.round(3))

# Visualize risk stratification
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Event rate by quintile
ax1 = axes[0, 0]
quintile_df.plot(x='Quintile', y='Event_Rate', kind='bar', ax=ax1, color='steelblue', alpha=0.7)
ax1.set_title('Event Rate by Risk Quintile')
ax1.set_ylabel('Event Rate')
ax1.grid(True, alpha=0.3)

# 2. Risk score distribution by quintile
ax2 = axes[0, 1]
for i, quintile in enumerate(['Q1', 'Q2', 'Q3', 'Q4', 'Q5']):
    mask = risk_quintiles == quintile
    quintile_scores = risk_scores[mask]
    ax2.hist(quintile_scores, bins=10, alpha=0.6, label=f'{quintile} (n={mask.sum()})', density=True)

ax2.set_xlabel('Risk Score')
ax2.set_ylabel('Density')
ax2.set_title('Risk Score Distribution by Quintile')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Time to event by quintile (for events only)
ax3 = axes[1, 0]
for i, quintile in enumerate(['Q1', 'Q2', 'Q3', 'Q4', 'Q5']):
    mask = risk_quintiles == quintile
    quintile_events = y_test.loc[mask, 'event'] == 1
    if quintile_events.sum() > 0:
        event_times = y_test.loc[mask & quintile_events, 'time_to_event']
        ax3.hist(event_times, bins=8, alpha=0.6, label=f'{quintile} (n={quintile_events.sum()})', density=True)

ax3.set_xlabel('Time to Event (days)')
ax3.set_ylabel('Density')
ax3.set_title('Time to Event Distribution by Quintile (Events Only)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Risk score vs time to event scatter
ax4 = axes[1, 1]
event_mask = y_test['event'] == 1
ax4.scatter(risk_scores[event_mask], y_test.loc[event_mask, 'time_to_event'], 
           alpha=0.6, c='red', label='Events')
ax4.scatter(risk_scores[~event_mask], y_test.loc[~event_mask, 'time_to_event'], 
           alpha=0.6, c='blue', label='No Events')

ax4.set_xlabel('Risk Score')
ax4.set_ylabel('Time to Event (days)')
ax4.set_title('Risk Score vs Time to Event')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nRisk Stratification Summary:")
print(f"Q1 (Lowest Risk): {quintile_df.iloc[0]['Event_Rate']:.1%} event rate")
print(f"Q5 (Highest Risk): {quintile_df.iloc[4]['Event_Rate']:.1%} event rate")
print(f"Risk Ratio (Q5/Q1): {quintile_df.iloc[4]['Event_Rate'] / quintile_df.iloc[0]['Event_Rate']:.1f}x")


In [ ]:
# Create risk groups based on Cox model predictions
risk_tertiles = pd.qcut(cox_risk_scores, q=3, labels=['Low', 'Medium', 'High'])
print(f"Risk group distribution:")
print(risk_tertiles.value_counts())

# Plot Kaplan-Meier curves by risk groups
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green', 'orange', 'red']
for i, group in enumerate(['Low', 'Medium', 'High']):
    mask = risk_tertiles == group
    
    # Get survival data for this group
    group_times = cox_test.loc[mask, 'time_to_event']
    group_events = cox_test.loc[mask, 'event']
    
    # Fit Kaplan-Meier
    kmf = KaplanMeierFitter()
    kmf.fit(group_times, group_events, label=f'{group} Risk (n={mask.sum()})')
    
    # Plot survival curve
    kmf.plot_survival_function(ax=ax, color=colors[i], linewidth=2)

ax.set_xlabel('Time to Readmission (days)')
ax.set_ylabel('Survival Probability (No Readmission)')
ax.set_title('Kaplan-Meier Survival Curves by Risk Groups')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

# Print survival statistics
print("\nSurvival Statistics by Risk Group:")
for group in ['Low', 'Medium', 'High']:
    mask = risk_tertiles == group
    group_events = cox_test.loc[mask, 'event']
    print(f"{group} Risk: {mask.sum()} patients, {group_events.sum()} events ({group_events.mean():.1%})")


## 3. Calibration Analysis


In [ ]:
# Convert risk scores to predicted probabilities for 30-day predictions
baseline_survival = cph.baseline_survival_
baseline_surv_30d = baseline_survival.iloc[baseline_survival.index <= 30].iloc[-1]
prob_30d = 1 - (baseline_surv_30d ** cox_risk_scores)

# Create calibration bins
n_bins = 10
bin_edges = np.linspace(0, 1, n_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

observed_rates = []
predicted_rates = []
bin_counts = []

for i in range(n_bins):
    mask = (prob_30d >= bin_edges[i]) & (prob_30d < bin_edges[i + 1])
    if i == n_bins - 1:  # Include the last edge
        mask = (prob_30d >= bin_edges[i]) & (prob_30d <= bin_edges[i + 1])
    
    if mask.sum() > 0:
        observed_rate = y_test.loc[mask, 'event'].mean()
        predicted_rate = prob_30d[mask].mean()
        
        observed_rates.append(observed_rate)
        predicted_rates.append(predicted_rate)
        bin_counts.append(mask.sum())
    else:
        observed_rates.append(np.nan)
        predicted_rates.append(np.nan)
        bin_counts.append(0)

# Plot calibration
plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.plot(predicted_rates, observed_rates, 'bo-', linewidth=2, markersize=6)
plt.plot([0, 1], [0, 1], 'r--', alpha=0.7, label='Perfect Calibration')
plt.xlabel('Predicted Probability')
plt.ylabel('Observed Rate')
plt.title('Calibration Plot (30-day)')
plt.grid(True, alpha=0.3)
plt.legend()

# Add sample sizes to points
for i, (pred, obs, count) in enumerate(zip(predicted_rates, observed_rates, bin_counts)):
    if not np.isnan(pred) and not np.isnan(obs):
        plt.annotate(f'n={count}', (pred, obs), xytext=(5, 5), textcoords='offset points', fontsize=8)

# Risk distribution
plt.subplot(2, 2, 2)
plt.hist(prob_30d, bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Predicted 30-day Risk')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Risks')
plt.grid(True, alpha=0.3)

# Calibration by risk groups
plt.subplot(2, 2, 3)
risk_groups = pd.qcut(prob_30d, q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
group_observed = []
group_predicted = []
group_labels = []

for group in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
    mask = risk_groups == group
    if mask.sum() > 0:
        obs_rate = y_test.loc[mask, 'event'].mean()
        pred_rate = prob_30d[mask].mean()
        group_observed.append(obs_rate)
        group_predicted.append(pred_rate)
        group_labels.append(f'{group}\n(n={mask.sum()})')

x_pos = np.arange(len(group_labels))
plt.bar(x_pos - 0.2, group_observed, 0.4, label='Observed', alpha=0.7)
plt.bar(x_pos + 0.2, group_predicted, 0.4, label='Predicted', alpha=0.7)
plt.xlabel('Risk Quintiles')
plt.ylabel('Event Rate')
plt.title('Calibration by Risk Quintiles')
plt.xticks(x_pos, group_labels)
plt.legend()
plt.grid(True, alpha=0.3)

# Model performance summary
plt.subplot(2, 2, 4)
plt.axis('off')
performance_text = f"""
Model Performance Summary:

C-index: {cox_c_index:.3f}
Mean Predicted Risk: {prob_30d.mean():.3f}
Observed Event Rate: {y_test['event'].mean():.3f}

Calibration:
• Well-calibrated if points follow diagonal
• Overconfident if above diagonal
• Underconfident if below diagonal
"""
plt.text(0.1, 0.5, performance_text, fontsize=10, verticalalignment='center')

plt.tight_layout()
plt.show()
